In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyomo.environ as pyo
import pyomo.opt as pyopt

from copt_pyomo import *

Cardinal Optimizer v5.0.4. Build date Aug 19 2022
Copyright Cardinal Operations 2022. All Rights Reserved



In [3]:
from evse.scoring import YearlyResult, get_scoring_dataframe

In [20]:
class Data:
    def __init__(self, path: str) -> None:    
        demand_path = path + "/forecast.csv"
        existingEV_path = path + "/existing_EV_infrastructure_2018.csv"
        self.df_dem = pd.read_csv(demand_path)
        self.df_dem = self.df_dem.set_index("demand_point_index", drop=True)
        self.df_ex = pd.read_csv(existingEV_path)
        self.df_ex = self.df_ex.set_index("supply_point_index", drop=True)
        
path = "data"
data = Data(path)

In [21]:
class OptModel(pyo.ConcreteModel):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def buildSets(self):
        self.i = pyo.Set(initialize=self.data.df_dem.index)
        self.j = pyo.Set(initialize=self.data.df_ex.index)
        self.k = pyo.Set(initialize=[2019, 2020])
        self.solver = pyopt.SolverFactory('copt_direct')
        self.solver.options["relgap"] = 1e-5
    
    def buildParams(self):
        def d_init(self, i, k):
            return self.data.df_dem.loc[i, f"{k}"]
        self.d = pyo.Param(self.i, self.k, initialize=d_init)
        
        def exSCS_init(self, j):
            return self.data.df_ex.loc[j, "existing_num_SCS"]
        self.exSCS = pyo.Param(self.j, initialize=exSCS_init)
        
        def exFCS_init(self, j):
            return self.data.df_ex.loc[j, "existing_num_FCS"]
        self.exFCS = pyo.Param(self.j, initialize=exFCS_init)
        
        def numPS_init(self, j):
            return self.data.df_ex.loc[j, "total_parking_slots"]
        self.numPS = pyo.Param(self.j, initialize=numPS_init)
        
        def dist_init(self, i ,j):
            return np.sqrt((self.data.df_ex.loc[j, "x_coordinate"] - self.data.df_dem.loc[i, "x_coordinate"]) ** 2.0 +
                           (self.data.df_ex.loc[j, "y_coordinate"] - self.data.df_dem.loc[i, "y_coordinate"]) ** 2.0)
        self.dist = pyo.Param(self.i, self.j, initialize=dist_init)
        
        self.capSCS = 200
        self.capFCS = 400
        
    def buildVars(self):
        self.ds = pyo.Var(self.i, self.j, self.k, domain=pyo.NonNegativeReals)
        self.insSCS = pyo.Var(self.j, self.k, domain=pyo.NonNegativeIntegers)
        self.insFCS = pyo.Var(self.j, self.k, domain=pyo.NonNegativeIntegers)

    def buildConstraints(self):
        def satDemCtr_rule(self, j, k):
            if k == 2019:
                existing_SCS = self.exSCS[j] + self.insSCS[j, 2019]
                existing_FCS = self.exFCS[j] + self.insFCS[j, 2019]
            else:
                existing_SCS = self.exSCS[j] + pyo.quicksum(self.insSCS[j, k1] for k1 in self.k)
                existing_FCS = self.exFCS[j] + pyo.quicksum(self.insFCS[j, k1] for k1 in self.k)
            ret = pyo.quicksum(self.ds[i, j, k] for i in self.i) <= \
                self.capSCS * existing_SCS + self.capFCS * existing_FCS - 1e-4
            return ret 
        self.satDemCtr = pyo.Constraint(self.j, self.k, rule=satDemCtr_rule)
        
        def forDemCtr_rule(self, i, k):
            ret = pyo.quicksum(self.ds[i, j, k] for j in self.j) == \
                self.d[i, k]
            return ret 
        self.forDemCtr = pyo.Constraint(self.i, self.k, rule=forDemCtr_rule) 
        
        def maxStaCtr_rule(self, j, k):
            if self.numPS[j] == 0:
                pyo.Constraint.Skip 
            if k == 2019:
                existing_SCS = self.exSCS[j] + self.insSCS[j, 2019]
                existing_FCS = self.exFCS[j] + self.insFCS[j, 2019]
            else:
                existing_SCS = self.exSCS[j] + pyo.quicksum(self.insSCS[j, k1] for k1 in self.k)
                existing_FCS = self.exFCS[j] + pyo.quicksum(self.insFCS[j, k1] for k1 in self.k)
            ret = existing_FCS + existing_SCS <= self.numPS[j]
            return ret 
        self.maxStaCtr = pyo.Constraint(self.j, self.k, rule=maxStaCtr_rule)
        
    def buildObjective(self):
        def objRule(self):
            return pyo.quicksum(self.dist[i, j] * self.ds[i, j, k] for k in self.k for j in self.j for i in self.i) \
                + 600 * pyo.quicksum(self.insSCS[j, k] + 1.5 * self.insFCS[j, k] for k in self.k for j in self.j)
        self.cost = pyo.Objective(rule=objRule, sense=pyo.minimize)
        
    def solve(self):
        self.results = self.solver.solve(self, tee=True) 

In [22]:
model = OptModel(data)
model.buildSets()
model.buildParams()
model.buildVars()
model.buildConstraints()
model.buildObjective()

In [23]:
model.solve()

Setting parameter 'RelGap' to 1e-05
Model fingerprint: a238320d

Hardware has 10 cores and 10 threads. Using instruction set ARMV8 (30)
Minimizing a MIP problem

The original problem has:
    8592 rows, 819600 columns and 1639600 non-zero elements
    400 integers

Presolving the problem

The presolved problem has:
    8237 rows, 784198 columns and 1568794 non-zero elements
    2 binaries and 396 integers

Starting the MIP solver with 10 threads and 32 tasks

     Nodes    Active  LPit/n  IntInf     BestBound  BestSolution    Gap   Time
         0         1      --       0  0.000000e+00            --    Inf  7.87s
         0         1      --      23  4.687721e+06            --    Inf 14.65s
H        0         1      --      23  4.687721e+06  4.697530e+06  0.21% 14.66s
         0         1      --      23  4.687721e+06  4.697530e+06  0.21% 19.40s
         0         1      --      21  4.687771e+06  4.697530e+06  0.21% 20.79s
H        0         1      --      21  4.687771e+06  4.696925e+

In [ ]:
class OptResults:
    def __init__(self, model):
        self.model = model
        
    def resultsToDf(self):
        years = [2019, 2020]
        names_tuples = [("value", s) for s in self.model.j]
        x_index = [d for d in self.model.i]
        cols_idx = pd.MultiIndex.from_tuples(names_tuples, names=[None, "supply_point_index"])
        
        for k in self.model.k:
            df_insSCS = pd.DataFrame(columns=["supply_point_index", "value"])
            df_insSCS = df_insSCS.set_index("supply_point_index")
            df_insFCS = pd.DataFrame(columns=["supply_point_index", "value"])
            df_insFCS = df_insFCS.set_index("supply_point_index")
            df_ds = pd.DataFrame(index=pd.Index(x_index, name="demand_point_index"), columns=cols_idx)
            
            for j in self.model.j:
                existingSCS = model.exSCS[j] 
                existingFCS = model.exFCS[j] 
                if k == 2019:
                    existingSCS += self.model.insSCS[j, k].value
                    existingFCS += self.model.insFCS[j, k].value
                else:
                    existingSCS += sum(self.model.insSCS[j, k1].value for k1 in self.model.k)
                    existingFCS += sum(self.model.insFCS[j, k1].value for k1 in self.model.k)
                df_insSCS.loc[j, "value"] = round(existingSCS)
                df_insFCS.loc[j, "value"] = round(existingFCS)    
                for i in self.model.i:
                    val = self.model.ds[i, j, k].value
                    df_ds.loc[i, "value"][j] = val if val >= 1e-6 else 0.0
            df_ds.to_csv(f"demand_supply_matrix_{k}.csv")   
            df_insSCS = df_insSCS.loc[:, "value"].astype(int)
            df_insSCS.to_csv(f"slow_charging_stations_on_supply_point_matrix_{k}.csv")
            df_insFCS = df_insFCS.loc[:, "value"].astype(int)
            df_insFCS.to_csv(f"fast_charging_stations_on_supply_point_matrix_{k}.csv")

In [ ]:
res = OptResults(model)
res.resultsToDf()

In [ ]:
given_yearly_results = [
            YearlyResult(
                year=k,
                slow_charging_stations_on_supply_point_matrix=pd.read_csv(
                    f"slow_charging_stations_on_supply_point_matrix_{k}.csv"
                ),
                fast_charging_stations_on_supply_point_matrix=pd.read_csv(
                    f"fast_charging_stations_on_supply_point_matrix_{k}.csv"
                ),
                demand_supply_matrix=pd.read_csv(
                    f"demand_supply_matrix_{k}.csv", header=[0, 1], index_col=[0]
                ),
            )
        for k in model.k]

# When
submission_dataframe = get_scoring_dataframe(given_yearly_results)

In [ ]:
submission_dataframe.to_csv("submission_MRC.csv")

In [189]:
#If infeasible model
from pyomo.util.infeasible import log_infeasible_constraints
import logging
model.write('model.lp', io_options={'symbolic_solver_labels':True})
log_infeasible_constraints(model, log_expression=True, log_variables=True)
logging.basicConfig(filename='infeas.log', encoding='utf-8', level=logging.INFO)
# # print(value(m.z))